# Imports:

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics.SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *

import os
from Utils import cacheOnDisk
from SegalQueue.qp import qp,fakeqp
from addloglevels import sethandlers

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
stop
done
done
done
done


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate


'10062018'

# def permanova function:
based on old function:

In [ ]:
the function permanova_withWrapper was copied to feature_phenotype_functions.py

# generate distMats:

## capped0.0001, trimmed, RA - genus/species, normed/not normed --> braycurtis

In [4]:
RAlist=['MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed','MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_normed',
       'MPA_g_standardParams_capped0_0001_percShared5_OLtrimmed','MPA_g_standardParams_capped0_0001_percShared5_OLtrimmed_normed']
metric='braycurtis'

for n,dfName in enumerate(RAlist):
    print n,dfName
    folder='%s/MicrobiomeDataTables/FilteredAndMergedOnBD' %MyPath
    file1='%s/%s' %(folder, dfName)
    df=pd.read_pickle(file1)

    df_condensed_org, distMat_square=genDistMat(df,metric)

    distMatfolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath

    RAdistMatFile='%s/%s_%s_distMat' %(distMatfolder,dfName,metric)
    RAdistMatFileCondensed='%s/%s_%s_distMat_CONDENSED' %(distMatfolder,dfName,metric)
    distMat_square.to_pickle(RAdistMatFile)
    df_condensed_org.to_pickle(RAdistMatFileCondensed)
    
    

0 MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed
generating condensed distance matrix using braycurtis
top similar pairs:
           dist sample1 sample2
34159  0.105432   BD334   BD419
5917   0.171121   BD120   BD905
25224  0.174412    BD24   BD334
52978  0.175495   BD419   BD533
25284  0.192836    BD24   BD419
34228  0.205078   BD334   BD533
25353  0.206463    BD24   BD533
61003  0.208095   BD457   BD533
25897  0.222987   BD245   BD893
70279  0.224804   BD533   BD905
generating squareform distance matrix for braycurtis
1 MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_normed
generating condensed distance matrix using braycurtis
top similar pairs:
           dist sample1 sample2
34159  0.122257   BD334   BD419
5917   0.171906   BD120   BD905
52978  0.174677   BD419   BD533
25224  0.191670    BD24   BD334
25284  0.194528    BD24   BD419
25353  0.204242    BD24   BD533
61003  0.209764   BD457   BD533
81941  0.215844    BD64   BD694
55464  0.217818   BD429   BD497
34228 

## capped0.0001, binary - genus/species, normed/not normed --> jaccard

In [5]:
RAlist=['MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_binary',
        'MPA_g_standardParams_capped0_0001_percShared5_OLtrimmed_binary']
metric='jaccard'

for n,dfName in enumerate(RAlist):
    print n,dfName
    folder='%s/MicrobiomeDataTables/FilteredAndMergedOnBD' %MyPath
    file1='%s/%s' %(folder, dfName)
    df=pd.read_pickle(file1)

    df_condensed_org, distMat_square=genDistMat(df,metric)

    distMatfolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath

    binaryDistMatFile='%s/%s_%s_distMat' %(distMatfolder,dfName,metric)
    binaryDistMatFileCondensed='%s/%s_%s_distMat_CONDENSED' %(distMatfolder,dfName,metric)
    distMat_square.to_pickle(binaryDistMatFile)
    df_condensed_org.to_pickle(binaryDistMatFileCondensed)
    
    

0 MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_binary
generating condensed distance matrix using jaccard
top similar pairs:
           dist sample1 sample2
81352  0.244186   BD631    BD71
22914  0.255814   BD228   BD849
82311  0.268817   BD645   BD852
65988  0.269231   BD491   BD802
80309  0.269663   BD617   BD838
82788  0.274725   BD650   BD923
4432   0.279070   BD113   BD457
85908  0.280000   BD701   BD881
61073  0.285714   BD457   BD650
64079  0.287234   BD480   BD765
generating squareform distance matrix for jaccard
1 MPA_g_standardParams_capped0_0001_percShared5_OLtrimmed_binary
generating condensed distance matrix using jaccard
top similar pairs:
           dist sample1 sample2
66008  0.111111   BD491   BD845
86184  0.117647   BD703   BD926
45363  0.128205   BD388   BD646
38334  0.131579   BD357   BD646
73713  0.135135   BD553   BD805
22914  0.135135   BD228   BD849
65923  0.138889   BD491   BD650
49533  0.138889   BD403   BD553
73603  0.138889   BD553   BD574
41967  0

# get phenotypeDF

In [6]:
dfName='MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed'
folder='%s/MicrobiomeDataTables/FilteredAndMergedOnBD' %MyPath
metric='braycurtis'




file1='%s/%s' %(folder, dfName)
df=pd.read_pickle(file1)

df_condensed_org, distMat_square=genDistMat(df,metric)

distMatfolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath

binaryDistMatFile='%s/%s_binary_%s_distMat' %(distMatfolder,dfName,metric)
binaryDistMatFileCondensed='%s/%s_binary_%s_distMat_CONDENSED' %(distMatfolder,dfName,metric)
distMat_square.to_pickle(binaryDistMatFile)
df_condensed_org.to_pickle(binaryDistMatFileCondensed)

generating condensed distance matrix using braycurtis
top similar pairs:
           dist sample1 sample2
34159  0.105432   BD334   BD419
5917   0.171121   BD120   BD905
25224  0.174412    BD24   BD334
52978  0.175495   BD419   BD533
25284  0.192836    BD24   BD419
34228  0.205078   BD334   BD533
25353  0.206463    BD24   BD533
61003  0.208095   BD457   BD533
25897  0.222987   BD245   BD893
70279  0.224804   BD533   BD905
generating squareform distance matrix for braycurtis


In [7]:
datasetFolder='%s/TCR_real_data/SubSampled9000data_rep2' %MyPath
datasetName='PNP515_ss9000_rep2'


file2='%s/sharingAnalysis/sharingMatrix_PNP515_ss9000_rep2_minNshared2_RA_onlyProductiveTrue__percShared20_OLtrimmed_binary' %datasetFolder
TCRdf=pd.read_pickle(file2)
TCRdf.head()

,CASSDRDTGELFF,CASSDSNTGELFF,CASSDSSGANVLTF,CASSDSSGSTDTQYF,CASSDSSTDTQYF,CASSELAGGPDTQYF,CASSGTSGSTDTQYF,CASSLAGGTDTQYF,CASSLAYEQYF,CASSLEETQYF,...,CASSLTDTQYF,CASSLYNEQFF,CASSPQETQYF,CASSPSTDTQYF,CASSPSYEQYF,CASSSQETQYF,CASSSSYEQYF,CASSSTDTQYF,CASSSYNEQFF,CASSYSYEQYF
Sample,,,,,,,,,,,,,,,,,,,,,
BD1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
BD10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
BD101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BD105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
BD106,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# find the most common sequence and take it to be Y

TCRdfSorted=TCRdf.copy()
TCRdfSorted.loc['sum',:]=TCRdfSorted.sum()
TCRdfSorted=TCRdfSorted.sort_values(by='sum',axis=1,ascending=False)
TCRdfSorted.tail()

,CASSLGETQYF,CASSLGYEQYF,CASSLQETQYF,CASSSSYEQYF,CASSLGGYEQYF,CASSLNTGELFF,CASSDRDTGELFF,CASSLSYEQYF,CASSLGGNTEAFF,CASSLEETQYF,...,CASSSYNEQFF,CASSDSSTDTQYF,CASSELAGGPDTQYF,CASSLGGNEQFF,CASSPQETQYF,CASSSQETQYF,CASSDSSGSTDTQYF,CASSLGTGELFF,CASSGTSGSTDTQYF,CASSLAGGTDTQYF
Sample,,,,,,,,,,,,,,,,,,,,,
BD944,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
BD945,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
BD948,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
BD949,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
sum,212.0,175.0,152.0,148.0,140.0,138.0,138.0,137.0,133.0,131.0,...,103.0,102.0,102.0,102.0,101.0,101.0,99.0,99.0,99.0,98.0


# Run permanova 


## phenotypeDF=TCR not capped, shared by 20% of the samples, MBfiles - capped at 0.0001, shared by 5%

In [9]:
print TCRdfSorted.shape

(491, 36)


In [10]:
phenotypeDF=pd.DataFrame(TCRdfSorted.iloc[:-1,:])
print phenotypeDF.shape

(490, 36)


In [11]:
folder='%s/MicrobiomeDataTables/distanceMatrices/forPermanova' %MyPath
distMats = [f for f in listdir(folder) if isfile(join(folder, f))]
count=0

nPerm=99
removeSameUser=True
permanovaDFfolder='%s/TCR_mb_results/seqTCRpermanova/dfs' %datasetFolder
if not isdir(permanovaDFfolder):
    makedirs(permanovaDFfolder)

for n, distMat in enumerate(distMats):
#     if n<1:
        print  n, distMat
        file1='%s/%s' %(folder,distMat)
        feature_distMat=pd.read_pickle(file1)
        feature_name=distMat

        for k,phenotypeColumn in enumerate(phenotypeDF):
            print count,k,phenotypeColumn
            permAnovaResDF=permanova_withWrapper(feature_distMat,feature_name,phenotypeDF,phenotypeColumn, 
                                         nPerm,removeSameUser,permanovaDFfolder)
            count=count+1

0 MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_braycurtis_distMat
0 0 CASSLGETQYF
conducting permanova...
0.98
1 1 CASSLGYEQYF
conducting permanova...
0.5
2 2 CASSLQETQYF
conducting permanova...
0.28
3 3 CASSSSYEQYF
conducting permanova...
0.39
4 4 CASSLGGYEQYF
conducting permanova...
0.69
5 5 CASSLNTGELFF
conducting permanova...
0.67
6 6 CASSDRDTGELFF
conducting permanova...
0.09
7 7 CASSLSYEQYF
conducting permanova...
0.18
8 8 CASSLGGNTEAFF
conducting permanova...
0.77
9 9 CASSLEETQYF
conducting permanova...
0.24
10 10 CASSDSSGANVLTF
conducting permanova...
0.24
11 11 CASSLRETQYF
conducting permanova...
0.02
12 12 CASSLGDTQYF
conducting permanova...
0.02
13 13 CASSPSYEQYF
conducting permanova...
0.66
14 14 CASSLAYEQYF
conducting permanova...
0.63
15 15 CASSDSNTGELFF
conducting permanova...
0.94
16 16 CASSLSSYEQYF
conducting permanova...
0.55
17 17 CASSYSYEQYF
conducting permanova...
0.08
18 18 CASSLYNEQFF
conducting permanova...
0.98
19 19 CASSLGQGYEQYF
conducting permanov

0.98
163 19 CASSLGQGYEQYF
conducting permanova...
0.16
164 20 CASSLSTDTQYF
conducting permanova...
0.04
165 21 CASSPSTDTQYF
conducting permanova...
0.94
166 22 CASSLNTEAFF
conducting permanova...
0.07
167 23 CASSLTDTQYF
conducting permanova...
0.62
168 24 CASSSTDTQYF
conducting permanova...
0.15
169 25 CASSLGTDTQYF
conducting permanova...
0.21
170 26 CASSSYNEQFF
conducting permanova...
0.74
171 27 CASSDSSTDTQYF
conducting permanova...
0.49
172 28 CASSELAGGPDTQYF
conducting permanova...
0.85
173 29 CASSLGGNEQFF
conducting permanova...
0.98
174 30 CASSPQETQYF
conducting permanova...
0.15
175 31 CASSSQETQYF
conducting permanova...
0.56
176 32 CASSDSSGSTDTQYF
conducting permanova...
0.78
177 33 CASSLGTGELFF
conducting permanova...
0.21
178 34 CASSGTSGSTDTQYF
conducting permanova...
0.41
179 35 CASSLAGGTDTQYF
conducting permanova...
0.08
5 MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_normed_braycurtis_distMat
180 0 CASSLGETQYF
conducting permanova...
0.98
181 1 CASSLGYEQYF
conduc

In [ ]:
131*8

### get results:

In [12]:
dfs_folder=permanovaDFfolder
PermanovaResults1=concat_summarizing_dfs(dfs_folder)

number of dfs in directory: 216
the length of df list is 216


In [13]:
PermanovaResults1.head()

,featureName,phenotype,nPerm,removeSameUser,s,p
0,MPA_s_standardParams_capped0_0001_percShared5_...,CASSELAGGPDTQYF,99.0,True,0.936852,0.55
0,MPA_g_standardParams_capped0_0001_percShared5_...,CASSLGYEQYF,99.0,True,0.991306,0.44
0,MPA_g_standardParams_capped0_0001_percShared5_...,CASSLGETQYF,99.0,True,1.353871,0.21
0,MPA_s_standardParams_capped0_0001_percShared5_...,CASSELAGGPDTQYF,99.0,True,0.935497,0.56
0,MPA_g_standardParams_capped0_0001_percShared5_...,CASSLAGGTDTQYF,99.0,True,1.313919,0.08


In [14]:
nTests=len(PermanovaResults1)
PermanovaResults1=PermanovaResults1.sort_values(by='p')
PermanovaResults1=add_corrected_pValues(PermanovaResults1,pValueColumn='p',nTests=nTests,FDR=0.1)
PermanovaResults1

,featureName,phenotype,nPerm,removeSameUser,s,p,Sig by bonferroni corrected pVal,sig. by FDR=0.1
0,MPA_g_standardParams_capped0_0001_percShared5_...,CASSLGDTQYF,99.0,True,2.563599,0.01,0,0
0,MPA_s_standardParams_capped0_0001_percShared5_...,CASSLGTGELFF,99.0,True,1.897652,0.01,0,0
0,MPA_s_standardParams_capped0_0001_percShared5_...,CASSLGDTQYF,99.0,True,2.078709,0.01,0,0
0,MPA_g_standardParams_capped0_0001_percShared5_...,CASSLGDTQYF,99.0,True,2.595249,0.01,0,0
0,MPA_g_standardParams_capped0_0001_percShared5_...,CASSLQETQYF,99.0,True,1.762418,0.02,0,0
0,MPA_s_standardParams_capped0_0001_percShared5_...,CASSLAGGTDTQYF,99.0,True,2.089608,0.02,0,0
0,MPA_s_standardParams_capped0_0001_percShared5_...,CASSLRETQYF,99.0,True,1.799447,0.02,0,0
0,MPA_s_standardParams_capped0_0001_percShared5_...,CASSLAGGTDTQYF,99.0,True,2.089168,0.02,0,0
0,MPA_g_standardParams_capped0_0001_percShared5_...,CASSLAGGTDTQYF,99.0,True,2.123325,0.02,0,0
0,MPA_s_standardParams_capped0_0001_percShared5_...,CASSDSSTDTQYF,99.0,True,1.769977,0.02,0,0


In [15]:
file2='%s/TCR_mb_results/seqTCRpermanova/PermanovaResults1.xlsx' %datasetFolder
PermanovaResults1.to_excel(file2)

## run permanova2 - results from permanova1 with p<=0.05, now with 99999 permutations

In [16]:
results005=PermanovaResults1[PermanovaResults1['p']<=0.05]

In [18]:
phenotypeList=results005['phenotype'].tolist()
featureNameList=results005['featureName'].tolist()

pairslist=zip(phenotypeList,featureNameList)
print pairslist[:3]
    

[('CASSLGDTQYF', 'MPA_g_standardParams_capped0_0001_percShared5_OLtrimmed_normed_braycurtis_distMat'), ('CASSLGTGELFF', 'MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_binary_jaccard_distMat'), ('CASSLGDTQYF', 'MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_normed_braycurtis_distMat')]


In [19]:
print len(pairslist)

19


In [22]:
folder='%s/MicrobiomeDataTables/distanceMatrices/forPermanova' %MyPath
count=0

nPerm=99999
removeSameUser=True
permanovaDFfolder='%s/TCR_mb_results/seqTCRpermanova/dfs2' %datasetFolder
if not isdir(permanovaDFfolder):
    makedirs(permanovaDFfolder)

for n, pair in enumerate(pairslist):
    phenotypeColumn=pair[0]
    feature=pair[1]
    print  n, pair
    file1='%s/%s' %(folder,feature)
    feature_distMat=pd.read_pickle(file1)
    feature_name=feature
   
    permAnovaResDF=permanova_withWrapper(feature_distMat,feature_name,phenotypeDF,phenotypeColumn, 
                                         nPerm,removeSameUser,permanovaDFfolder)
    count=count+1
    

0 ('CASSLGDTQYF', 'MPA_g_standardParams_capped0_0001_percShared5_OLtrimmed_normed_braycurtis_distMat')
conducting permanova...
0.0099
1 ('CASSLGTGELFF', 'MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_binary_jaccard_distMat')
conducting permanova...
0.00255
2 ('CASSLGDTQYF', 'MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_normed_braycurtis_distMat')
conducting permanova...
0.00958
3 ('CASSLGDTQYF', 'MPA_g_standardParams_capped0_0001_percShared5_OLtrimmed_braycurtis_distMat')
conducting permanova...
0.00905
4 ('CASSLQETQYF', 'MPA_g_standardParams_capped0_0001_percShared5_OLtrimmed_binary_jaccard_distMat')
conducting permanova...
0.02861
5 ('CASSLAGGTDTQYF', 'MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_normed_braycurtis_distMat')
conducting permanova...
0.00843
6 ('CASSLRETQYF', 'MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_braycurtis_distMat')
conducting permanova...
0.02544
7 ('CASSLAGGTDTQYF', 'MPA_s_standardParams_capped0_0001_percShared5_OLtr

### get results:

In [ ]:
datasetFolder='%s/TCR_real_data/SubSampled9000data_rep2' %MyPath
datasetName='PNP515_ss9000_rep2'

dfs_folder='%s/TCR_mb_results/seqTCRpermanova/dfs2' %datasetFolder
PermanovaResults2=concat_summarizing_dfs(dfs_folder)

In [ ]:
PermanovaResults2.head()

In [ ]:
nTests=len(PermanovaResults2)
PermanovaResults2=PermanovaResults2.sort_values(by='p')
PermanovaResults2=add_corrected_pValues(PermanovaResults2,pValueColumn='p',nTests=nTests,FDR=0.1)
PermanovaResults2

## merge two permanova results

In [ ]:
file2='%s/TCR_mb_results/seqTCRpermanova/PermanovaResults1.xlsx' %datasetFolder
PermanovaResults1=pd.read_excel(file2)

In [ ]:
mergedPA=pd.merge(PermanovaResults1,PermanovaResults2,how='left', left_on=('featureName','phenotype'),
                 right_on=('featureName','phenotype'))
for column in mergedPA.columns:
    newName=column.replace('_x','_PA99')
    newName=newName.replace('_y','_PA99999')
    mergedPA=mergedPA.rename(columns={column:newName})
print len(PermanovaResults1)
print len(PermanovaResults2)
print len(mergedPA)
mergedPA.head()

In [ ]:
file1='%s/TCR_mb_results/mergedResults_top279pairsquickPerms9999.xlsx' %datasetFolder
old=pd.read_excel(file1)

In [ ]:
old=old.sort_values(by=['permP_corr','permP_fisher'])
for column in old.columns:
    newName=column.replace('_x','_corr')
    newName=newName.replace('_y','_fisher')
    old=old.rename(columns={column:newName})
old['rank']=range(1,len(old)+1)
old.head()
        

In [ ]:
oldGrouped=old.groupby('seq').agg({'species': lambda x:list(x),'rank':'min',
                                  'permP_corr':'min','Sig by bonferroni corrected pVal_corr':'max',
                                  'sig. by FDR=0.1_corr':'max','permP_fisher':'min','Sig by bonferroni corrected pVal_fisher':'max',
                                  'sig. by FDR=0.1_fisher':'max','Epitope species_VDJDB':lambda x:list(x),
                                  'Epitope species_CMVpaper':lambda x:list(x),'Pathology_McPAS':lambda x:list(x) })
oldGrouped

In [ ]:
mergedPAwithOldTests=pd.merge(mergedPA,oldGrouped, how='left', left_on='phenotype',right_index=True)


print len(mergedPA)
print len(oldGrouped)
print len(mergedPAwithOldTests)

mergedPAwithOldTests=mergedPAwithOldTests.sort_values(by='p_PA99')
mergedPAwithOldTests.head(10)

In [ ]:
file2='%s/TCR_mb_results/seqTCRpermanova/PermanovaResults2_withInfo.xlsx' %datasetFolder
mergedPAwithOldTests.to_excel(file2)

# define 2-steps permanova function:

In [ ]:
#input:

In [ ]:
import lightgbm as lgb